In [1]:
import tensorflow.keras as k
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import neural_net_project_auxiliary as nn
DM = nn.DataManipulator
import plotly.graph_objs as go
from ipywidgets import interact, interactive, FloatSlider, VBox
import ipywidgets as widgets
from IPython.display import display, clear_output
%matplotlib widget

2026-01-30 09:29:13.482618: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-30 09:29:13.544960: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-30 09:29:13.547019: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-30 09:29:15.035552: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/srv/conda/envs/notebook/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Pleas

In [2]:
os.chdir("/")
for root, dirs, files in os.walk('.'):
    if 'models' in dirs:
        found_dir_path = os.path.join(root, 'models')
os.chdir(found_dir_path)

model_abs_77K = tf.keras.models.load_model('final_abs_spectra_from_params_7_pigs_dis_120_77K', compile=False)
model_abs_290K = tf.keras.models.load_model('final_abs_spectra_from_params_7_pigs_dis_120_290K', compile=False)
model_fl_77K = tf.keras.models.load_model('final_fl_spectra_from_params_7_pigs_dis_120_290K', compile=False)
model_fl_290K = tf.keras.models.load_model('final_fl_spectra_from_params_7_pigs_dis_120_290K', compile=False)
model_array = np.array([[model_abs_77K, model_abs_290K], [model_fl_77K, model_fl_290K]])

In [3]:
os.chdir("/")
for root, dirs, files in os.walk('.'):
    if 'statistical_props' in dirs:
        found_dir_path = os.path.join(root, 'statistical_props')
os.chdir(found_dir_path)

par_means, par_stds, spec_means, spec_stds,  = np.empty((2,2), dtype=object), np.empty((2,2), dtype=object),\
                                               np.empty((2,2), dtype=object), np.empty((2,2), dtype=object)
for ss, spec in enumerate(['abs', 'fl']):
    for tt, temp in enumerate(['77K', '290K']): 
        par_means[ss, tt] = np.load(f'params_mean_7pigs_dis120_{spec}_{temp}.npy')
        spec_means[ss, tt] = np.load(f'spectra_mean_7pigs_dis120_{spec}_{temp}.npy')
        par_stds[ss, tt] = np.load(f'params_std_7pigs_dis120_{spec}_{temp}.npy')
        spec_stds[ss, tt] = np.load(f'spectra_std_7pigs_dis120_{spec}_{temp}.npy')

# ANN-predicted linear spectra

Shown below are the ANN-predicted, inhomogeneously-broadened absorption and fluorescence spectra of a model complex that has excitonic couplings and dipole moments of the Fenna-Matthews-Olson light-harvesting complex. The sliders can be used to change the site energies. As an illustration, the last slider controls the coupling between pigments 1 and 2. After downloading from GitHub, this script can be adapted to investigate spectral changes brought about by changes in the dipole moments, other excitonic couplings, or dipole factor (the latter determines whether absorption, linear dichroism, or circular dichroism is calculated). For more information, see Nothling, J.A. and Krüger, T.P.J. Predicting spectra and molecular parameters of molecular aggregates using artificial neural network models. 2026. In preparation.

In [9]:
os.chdir("/")
for root, dirs, files in os.walk('.'):
    if 'statistical_props' in dirs:
        found_dir_path = os.path.join(root, 'statistical_props')
os.chdir(found_dir_path)
os.chdir("../")
freq_axis_abs = np.load('abs_frequency_axis.npy')
freq_axis_fl = np.load('fl_frequency_axis.npy')
Ham_abs = np.load('FMO_Ham.npy')
Ham_fl = np.load('FMO_Ham.npy')
dipole_moments = np.load('FMO_3eni_dmoms.npy')
site_dip_factor = dipole_moments@dipole_moments.T
site_dip_factor_max = np.amax(np.abs(site_dip_factor))
site_dip_factor /= site_dip_factor_max
number_of_pigments  = 7
bounds_lo = [0,0,0,0,0,0,0,-55]
bounds_hi = [1400,1400,1400,1400,1400,1400,1400,55]
slider_desc = ["site energy 1", "site energy 2", "site energy 3", "site energy 4", "site energy 5", "site energy 6", "site energy 7", "coupling 12"]


def make_sliders(bounds_lo, bounds_hi):
    return [
        widgets.FloatSlider(
            value=(bounds_lo[i] + bounds_hi[i])/2,
            min=bounds_lo[i],
            max=bounds_hi[i],
            step=(bounds_hi[i] - bounds_lo[i])/100,
            description=slider_desc[i],
            continuous_update=True
        )
        for i in range(8)
    ]

fig1 = go.FigureWidget(
    data=[go.Scatter(x=freq_axis_abs, y=np.zeros_like(freq_axis_abs), mode="lines", name='290 K'),
          go.Scatter(x=freq_axis_abs, y=np.zeros_like(freq_axis_abs), mode="lines", name="77 K"),],
    
    layout=go.Layout(title="ANN-predicted absorption spectra",
                    xaxis_title="Energy (per cm)",
                    yaxis_title="Absorption (a.u.)")
)

sliders1 = make_sliders(bounds_lo, bounds_hi)

def update_fig1(change=None):
    slider_vals_abs = [s.value for s in sliders1]
    Ham_abs[np.diag_indices_from(Ham_abs)] = slider_vals_abs[:-1]
    Ham_abs[0,1] = slider_vals_abs[-1]
    Ham_abs[1,0] = slider_vals_abs[-1]
    exciton_ens, SS = np.linalg.eigh(Ham_abs)     
    gamma_aamm = (SS**2).T@(SS**2)
    C_coeffs = SS.flatten()                        
    ex_dip_factor = np.diag((SS.T)@site_dip_factor@SS)
    params_abs = np.concatenate((exciton_ens,\
                         (gamma_aamm[np.triu_indices_from(gamma_aamm)]).flatten(),\
                           ex_dip_factor, C_coeffs))
    
    params_290_abs = DM.stat_normalise(params_abs,  mean=par_means[0,1], std=par_stds[0,1])
    predicted_spec_290_abs = DM.stat_denormalise(model_array[0, 1].predict(np.array([params_290_abs]), verbose=0)[0], mean=spec_means[0,1], std=spec_stds[0,1])   

    params_77_abs = DM.stat_normalise(params_abs,  mean=par_means[0,0], std=par_stds[0,0])
    predicted_spec_77_abs = DM.stat_denormalise(model_array[0, 0].predict(np.array([params_77_abs]), verbose=0)[0], mean=spec_means[0,0], std=spec_stds[0,0])    

    
    with fig1.batch_update():
        fig1.data[0].y = predicted_spec_290_abs
        fig1.data[1].y = predicted_spec_77_abs

for s in sliders1:
    s.observe(update_fig1, names="value")

update_fig1()


fig2 = go.FigureWidget(
    data=[go.Scatter(x=freq_axis_fl, y=np.zeros_like(freq_axis_fl), mode="lines", name="290 K"),
          go.Scatter(x=freq_axis_fl, y=np.zeros_like(freq_axis_fl), mode="lines", name="77 K"),],
    layout=go.Layout(title="ANN-predicted fluorescence spectra",
                            xaxis_title="Energy (per cm)",
                            yaxis_title="Fluorescence (a.u.)")
)

sliders2 = make_sliders(bounds_lo, bounds_hi)

def update_fig2(change=None):
    slider_vals_fl = [s.value for s in sliders2]
    Ham_fl[np.diag_indices_from(Ham_fl)] = slider_vals_fl[:-1]
    Ham_fl[0,1] = slider_vals_fl[-1]
    Ham_fl[1,0] = slider_vals_fl[-1]
    exciton_ens, SS = np.linalg.eigh(Ham_fl)     
    gamma_aamm = (SS**2).T@(SS**2)
    C_coeffs = SS.flatten()                        
    ex_dip_factor = np.diag((SS.T)@site_dip_factor@SS)
    params_fl = np.concatenate((exciton_ens,\
                         (gamma_aamm[np.triu_indices_from(gamma_aamm)]).flatten(),\
                           ex_dip_factor, C_coeffs))
    params_290_fl = DM.stat_normalise(params_fl,  mean=par_means[1,1], std=par_stds[1,1])
    predicted_spec_290_fl = DM.stat_denormalise(model_array[1, 1].predict(np.array([params_290_fl]), verbose=0)[0], mean=spec_means[1,1], std=spec_stds[1,1])   

    params_77_fl = DM.stat_normalise(params_fl,  mean=par_means[1,0], std=par_stds[1,0])
    predicted_spec_77_fl = DM.stat_denormalise(model_array[1, 0].predict(np.array([params_77_fl]), verbose=0)[0], mean=spec_means[1,0], std=spec_stds[1,0])    
   
    with fig2.batch_update():
        fig2.data[0].y = predicted_spec_290_fl
        fig2.data[1].y = predicted_spec_77_fl

for s in sliders2:
    s.observe(update_fig2, names="value")

update_fig2()

display(
    widgets.HBox([
        widgets.VBox(sliders1 + [fig1]),
        widgets.VBox(sliders2 + [fig2])
    ])
)